In [ ]:
#!pip install fastener

In [1]:
# import preprocessing tools
from sklearn import preprocessing
import numpy as np
import pandas as pd

# import learning/evaluation
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

# typing
from typing import Dict, List, Callable, Any, Tuple, Optional, \
    Counter as CounterType, Set

# FASTENER specific imports
from fastener.random_utils import shuffle
from fastener.item import Result, Genes, RandomFlipMutationStrategy, RandomEveryoneWithEveryone, \
    IntersectionMating, UnionMating, IntersectionMatingWithInformationGain, \
    IntersectionMatingWithWeightedRandomInformationGain
from fastener import fastener

In [2]:
# prepare data
from data import *

# load data
static_df = load_static()
weather_df = pd.read_pickle('data/weather/weather.pkl')
# load sensor - last parameter is id
final_df = make_final(static_df, weather_df, 1)

def make_target(df, h):
    y = df.shift(periods=-h, freq='1H')["pc"]
    df.pop('pc15')
    df.pop('pc30')
    df.pop('pc45')
    y = y[h:]
    X = df[:-h]
    return X, y

X_df, y_df = make_target(final_df, 1)

# basic dataset split
n_sample = X_df.shape[0]
n_test = int(n_sample * 0.8)

labels_train = y_df.to_numpy()[:n_test]
labels_test = y_df.to_numpy()[n_test:]

XX_train = X_df.to_numpy()[:n_test, :]
XX_test = X_df.to_numpy()[n_test:, :]

In [3]:
# define evaluation function
def eval_fun(model: Any, genes: "Genes", shuffle_indices: Optional[List[int]] = None) -> "Result":
    test_data = XX_test[:, genes]
    if shuffle_indices:
        test_data = test_data.copy()
        for j in shuffle_indices:
            shuffle(test_data[:, j])
    pred = model.predict(test_data)
    res = Result(r2_score(labels_test, pred))
    return res

In [ ]:
number_of_genes = XX_train.shape[1]
general_model = DecisionTreeRegressor
# output folder name must be changed every time the algorithm is run
output_folder_name = "test"

# to start the algorithm initial_genes or initial_population must be provided
initial_genes = [
    [398]
]

# Select mating selection strategie (RandomEveryoneWithEveryone, NoMating) and mating strategy
# (UnionMating, IntersectionMating, IntersectionMatingWithInformationGain, 
# IntersectionMatingWithWeightedRandomInformationGain) 
# If regression model is used IntersectionMatingWithInformationGain, IntersectionMatingWithWeightedRandomInformationGain 
# must have regression=True set (eg. IntersectionMatingWithInformationGain(regression=True))
mating = RandomEveryoneWithEveryone(pool_size=3, mating_strategy=IntersectionMatingWithWeightedRandomInformationGain(regression=True))

# Random mutation (probability of gene mutating: 1 / number_of_genes)
mutation = RandomFlipMutationStrategy(1 / number_of_genes)

entropy_optimizer = fastener.EntropyOptimizer(
    general_model, XX_train, labels_train, eval_fun,
    number_of_genes, mating, mutation, initial_genes=initial_genes,
    config=fastener.Config(output_folder=output_folder_name, random_seed=2020, reset_to_pareto_rounds=5, number_of_rounds=500)
)

In [ ]:
# fastener loop
entropy_optimizer.mainloop()

In [ ]:
# check evaluation
# read log from last generation
object = pd.read_pickle(f'log/{output_folder_name}/generation_20.pickle')

# list of best-scoring EvalItem objects for each number of features
best = list(object.front.values())

for item in best:
    # names of best features
    selected_features = final_df.iloc[:, item.genes].columns.tolist()

    X = X_df[selected_features].values.astype(float)
    y = y_df.values.astype(float)

    # evaluates each set of features with cross validation
    model = DecisionTreeRegressor()
    cvs = cross_val_score(model, X, y, cv=10)
    print("Features:", selected_features)
    print("Accuracy: ", cvs.mean(), " stdev: ", cvs.std(), "\n")

In [ ]:
best

In [ ]:
list(final_df.columns).index("pc")

In [4]:
# run a series of experiments

horizons = [1, 3, 6, 12, 24, 36]
sensors = [312, 386, 235, 368, 290, 1]
# sensors = [1, 24, 35, 43, 44, 68, 103, 105, 106, 108, 109, 110, 116, 117, 122, 124, 144, 145, 157, 168, 177, 179, 223,
# 225, 236, 239, 243, 245, 249, 253, 257, 259, 262, 285, 286, 290, 293, 301, 303, 312, 333, 358, 359, 362, 365, 367, 377,
# 378, 379, 382, 384, 385, 388, 389, 391, 392, 394, 395]

sensors = [312, 333, 358, 359, 362, 365, 367, 377,
 378, 379, 382, 384, 385, 388, 389, 391, 392, 394, 395]


for s in sensors:
    for h in horizons:
        output = "s{}h{}".format(s, h)
        print("\n\n *** {} *** \n".format(output))
        
        
        # load data
        # load data
        static_df = load_static()
        weather_df = pd.read_pickle('data/weather/weather.pkl')
        # load sensor - last parameter is id
        final_df = make_final(static_df, weather_df, s)

        X_df, y_df = make_target(final_df, h)

        # basic dataset split
        n_sample = X_df.shape[0]
        n_test = int(n_sample * 0.8)

        labels_train = y_df.to_numpy()[:n_test]
        labels_test = y_df.to_numpy()[n_test:]

        XX_train = X_df.to_numpy()[:n_test, :]
        XX_test = X_df.to_numpy()[n_test:, :]
        
        number_of_genes = XX_train.shape[1]
        general_model = DecisionTreeRegressor
        # output folder name must be changed every time the algorithm is run
        output_folder_name = output

        # to start the algorithm initial_genes or initial_population must be provided
        initial_genes = [
            [0]
        ]

        # Select mating selection strategie (RandomEveryoneWithEveryone, NoMating) and mating strategy
        # (UnionMating, IntersectionMating, IntersectionMatingWithInformationGain, 
        # IntersectionMatingWithWeightedRandomInformationGain) 
        # If regression model is used IntersectionMatingWithInformationGain, IntersectionMatingWithWeightedRandomInformationGain 
        # must have regression=True set (eg. IntersectionMatingWithInformationGain(regression=True))
        mating = RandomEveryoneWithEveryone(pool_size=3, mating_strategy=IntersectionMatingWithWeightedRandomInformationGain(regression=True))

        # Random mutation (probability of gene mutating: 1 / number_of_genes)
        mutation = RandomFlipMutationStrategy(1 / number_of_genes)

        entropy_optimizer = fastener.EntropyOptimizer(
            general_model, XX_train, labels_train, eval_fun,
            number_of_genes, mating, mutation, initial_genes=initial_genes,
            config=fastener.Config(output_folder=output_folder_name, random_seed=2020, reset_to_pareto_rounds=5, number_of_rounds=1000)
        )
        # change number of rounds to 1000 for final evaluation
        entropy_optimizer.mainloop()



 *** s312h1 *** 

Round: 1
Round: 2
Round: 3
Round: 4
Round: 5
Round: 6
Round: 7
Round: 8
Round: 9
Round: 10
Round: 11
Round: 12
Round: 13
Round: 14
Round: 15
Round: 16
Round: 17
Round: 18
Round: 19
Round: 20
Round: 21
Round: 22
Round: 23
Round: 24
Round: 25
Round: 26
Round: 27
Round: 28
Round: 29
Round: 30
Round: 31
Round: 32
Round: 33
Round: 34
Round: 35
Round: 36
Round: 37
Round: 38
Round: 39
Round: 40
Round: 41
Round: 42
Round: 43
Round: 44
Round: 45
Round: 46
Round: 47
Round: 48
Round: 49
Round: 50
Round: 51
Round: 52
Round: 53
Round: 54
Round: 55
Round: 56
Round: 57
Round: 58
Round: 59
Round: 60
Round: 61
Round: 62
Round: 63
Round: 64
Round: 65
Round: 66
Round: 67
Round: 68
Round: 69
Round: 70
Round: 71
Round: 72
Round: 73
Round: 74
Round: 75
Round: 76
Round: 77
Round: 78
Round: 79
Round: 80
Round: 81
Round: 82
Round: 83
Round: 84
Round: 85
Round: 86
Round: 87
Round: 88
Round: 89
Round: 90
Round: 91
Round: 92
Round: 93
Round: 94
Round: 95
Round: 96
Round: 97
Round: 98
Round: 99

Round: 754
Round: 755
Round: 756
Round: 757
Round: 758
Round: 759
Round: 760
Round: 761
Round: 762
Round: 763
Round: 764
Round: 765
Round: 766
Round: 767
Round: 768
Round: 769
Round: 770
Round: 771
Round: 772
Round: 773
Round: 774
Round: 775
Round: 776
Round: 777
Round: 778
Round: 779
Round: 780
Round: 781
Round: 782
Round: 783
Round: 784
Round: 785
Round: 786
Round: 787
Round: 788
Round: 789
Round: 790
Round: 791
Round: 792
Round: 793
Round: 794
Round: 795
Round: 796
Round: 797
Round: 798
Round: 799
Round: 800
Round: 801
Round: 802
Round: 803
Round: 804
Round: 805
Round: 806
Round: 807
Round: 808
Round: 809
Round: 810
Round: 811
Round: 812
Round: 813
Round: 814
Round: 815
Round: 816
Round: 817
Round: 818
Round: 819
Round: 820
Round: 821
Round: 822
Round: 823
Round: 824
Round: 825
Round: 826
Round: 827
Round: 828
Round: 829
Round: 830
Round: 831
Round: 832
Round: 833
Round: 834
Round: 835
Round: 836
Round: 837
Round: 838
Round: 839
Round: 840
Round: 841
Round: 842
Round: 843
Round: 844

Round: 507
Round: 508
Round: 509
Round: 510
Round: 511
Round: 512
Round: 513
Round: 514
Round: 515
Round: 516
Round: 517
Round: 518
Round: 519
Round: 520
Round: 521
Round: 522
Round: 523
Round: 524
Round: 525
Round: 526
Round: 527
Round: 528
Round: 529
Round: 530
Round: 531
Round: 532
Round: 533
Round: 534
Round: 535
Round: 536
Round: 537
Round: 538
Round: 539
Round: 540
Round: 541
Round: 542
Round: 543
Round: 544
Round: 545
Round: 546
Round: 547
Round: 548
Round: 549
Round: 550
Round: 551
Round: 552
Round: 553
Round: 554
Round: 555
Round: 556
Round: 557
Round: 558
Round: 559
Round: 560
Round: 561
Round: 562
Round: 563
Round: 564
Round: 565
Round: 566
Round: 567
Round: 568
Round: 569
Round: 570
Round: 571
Round: 572
Round: 573
Round: 574
Round: 575
Round: 576
Round: 577
Round: 578
Round: 579
Round: 580
Round: 581
Round: 582
Round: 583
Round: 584
Round: 585
Round: 586
Round: 587
Round: 588
Round: 589
Round: 590
Round: 591
Round: 592
Round: 593
Round: 594
Round: 595
Round: 596
Round: 597

Round: 260
Round: 261
Round: 262
Round: 263
Round: 264
Round: 265
Round: 266
Round: 267
Round: 268
Round: 269
Round: 270
Round: 271
Round: 272
Round: 273
Round: 274
Round: 275
Round: 276
Round: 277
Round: 278
Round: 279
Round: 280
Round: 281
Round: 282
Round: 283
Round: 284
Round: 285
Round: 286
Round: 287
Round: 288
Round: 289
Round: 290
Round: 291
Round: 292
Round: 293
Round: 294
Round: 295
Round: 296
Round: 297
Round: 298
Round: 299
Round: 300
Round: 301
Round: 302
Round: 303
Round: 304
Round: 305
Round: 306
Round: 307
Round: 308
Round: 309
Round: 310
Round: 311
Round: 312
Round: 313
Round: 314
Round: 315
Round: 316
Round: 317
Round: 318
Round: 319
Round: 320
Round: 321
Round: 322
Round: 323
Round: 324
Round: 325
Round: 326
Round: 327
Round: 328
Round: 329
Round: 330
Round: 331
Round: 332
Round: 333
Round: 334
Round: 335
Round: 336
Round: 337
Round: 338
Round: 339
Round: 340
Round: 341
Round: 342
Round: 343
Round: 344
Round: 345
Round: 346
Round: 347
Round: 348
Round: 349
Round: 350

Round: 4
Round: 5
Round: 6
Round: 7
Round: 8
Round: 9
Round: 10
Round: 11
Round: 12
Round: 13
Round: 14
Round: 15
Round: 16
Round: 17
Round: 18
Round: 19
Round: 20
Round: 21
Round: 22
Round: 23
Round: 24
Round: 25
Round: 26
Round: 27
Round: 28
Round: 29
Round: 30
Round: 31
Round: 32
Round: 33
Round: 34
Round: 35
Round: 36
Round: 37
Round: 38
Round: 39
Round: 40
Round: 41
Round: 42
Round: 43
Round: 44
Round: 45
Round: 46
Round: 47
Round: 48
Round: 49
Round: 50
Round: 51
Round: 52
Round: 53
Round: 54
Round: 55
Round: 56
Round: 57
Round: 58
Round: 59
Round: 60
Round: 61
Round: 62
Round: 63
Round: 64
Round: 65
Round: 66
Round: 67
Round: 68
Round: 69
Round: 70
Round: 71
Round: 72
Round: 73
Round: 74
Round: 75
Round: 76
Round: 77
Round: 78
Round: 79
Round: 80
Round: 81
Round: 82
Round: 83
Round: 84
Round: 85
Round: 86
Round: 87
Round: 88
Round: 89
Round: 90
Round: 91
Round: 92
Round: 93
Round: 94
Round: 95
Round: 96
Round: 97
Round: 98
Round: 99
Round: 100
Round: 101
Round: 102
Round: 103
Ro

...

Round: 126
Round: 127
Round: 128
Round: 129
Round: 130
Round: 131
Round: 132
Round: 133
Round: 134
Round: 135
Round: 136
Round: 137
Round: 138
Round: 139
Round: 140
Round: 141
Round: 142
Round: 143
Round: 144
Round: 145
Round: 146
Round: 147
Round: 148
Round: 149
Round: 150
Round: 151
Round: 152
Round: 153
Round: 154
Round: 155
Round: 156
Round: 157
Round: 158
Round: 159
Round: 160
Round: 161
Round: 162
Round: 163
Round: 164
Round: 165
Round: 166
Round: 167
Round: 168
Round: 169
Round: 170
Round: 171
Round: 172
Round: 173
Round: 174
Round: 175
Round: 176
Round: 177
Round: 178
Round: 179
Round: 180
Round: 181
Round: 182
Round: 183
Round: 184
Round: 185
Round: 186
Round: 187
Round: 188
Round: 189
Round: 190
Round: 191
Round: 192
Round: 193
Round: 194
Round: 195
Round: 196
Round: 197
Round: 198
Round: 199
Round: 200
Round: 201
Round: 202
Round: 203
Round: 204
Round: 205
Round: 206
Round: 207
Round: 208
Round: 209
Round: 210
Round: 211
Round: 212
Round: 213
Round: 214
Round: 215
Round: 216

Round: 871
Round: 872
Round: 873
Round: 874
Round: 875
Round: 876
Round: 877
Round: 878
Round: 879
Round: 880
Round: 881
Round: 882
Round: 883
Round: 884
Round: 885
Round: 886
Round: 887
Round: 888
Round: 889
Round: 890
Round: 891
Round: 892
Round: 893
Round: 894
Round: 895
Round: 896
Round: 897
Round: 898
Round: 899
Round: 900
Round: 901
Round: 902
Round: 903
Round: 904
Round: 905
Round: 906
Round: 907
Round: 908
Round: 909
Round: 910
Round: 911
Round: 912
Round: 913
Round: 914
Round: 915
Round: 916
Round: 917
Round: 918
Round: 919
Round: 920
Round: 921
Round: 922
Round: 923
Round: 924
Round: 925
Round: 926
Round: 927
Round: 928
Round: 929
Round: 930
Round: 931
Round: 932
Round: 933
Round: 934
Round: 935
Round: 936
Round: 937
Round: 938
Round: 939
Round: 940
Round: 941
Round: 942
Round: 943
Round: 944
Round: 945
Round: 946
Round: 947
Round: 948
Round: 949
Round: 950
Round: 951
Round: 952
Round: 953
Round: 954
Round: 955
Round: 956
Round: 957
Round: 958
Round: 959
Round: 960
Round: 961

Round: 624
Round: 625
Round: 626
Round: 627
Round: 628
Round: 629
Round: 630
Round: 631
Round: 632
Round: 633
Round: 634
Round: 635
Round: 636
Round: 637
Round: 638
Round: 639
Round: 640
Round: 641
Round: 642
Round: 643
Round: 644
Round: 645
Round: 646
Round: 647
Round: 648
Round: 649
Round: 650
Round: 651
Round: 652
Round: 653
Round: 654
Round: 655
Round: 656
Round: 657
Round: 658
Round: 659
Round: 660
Round: 661
Round: 662
Round: 663
Round: 664
Round: 665
Round: 666
Round: 667
Round: 668
Round: 669
Round: 670
Round: 671
Round: 672
Round: 673
Round: 674
Round: 675
Round: 676
Round: 677
Round: 678
Round: 679
Round: 680
Round: 681
Round: 682
Round: 683
Round: 684
Round: 685
Round: 686
Round: 687
Round: 688
Round: 689
Round: 690
Round: 691
Round: 692
Round: 693
Round: 694
Round: 695
Round: 696
Round: 697
Round: 698
Round: 699
Round: 700
Round: 701
Round: 702
Round: 703
Round: 704
Round: 705
Round: 706
Round: 707
Round: 708
Round: 709
Round: 710
Round: 711
Round: 712
Round: 713
Round: 714

Round: 377
Round: 378
Round: 379
Round: 380
Round: 381
Round: 382
Round: 383
Round: 384
Round: 385
Round: 386
Round: 387
Round: 388
Round: 389
Round: 390
Round: 391
Round: 392
Round: 393
Round: 394
Round: 395
Round: 396
Round: 397
Round: 398
Round: 399
Round: 400
Round: 401
Round: 402
Round: 403
Round: 404
Round: 405
Round: 406
Round: 407
Round: 408
Round: 409
Round: 410
Round: 411
Round: 412
Round: 413
Round: 414
Round: 415
Round: 416
Round: 417
Round: 418
Round: 419
Round: 420
Round: 421
Round: 422
Round: 423
Round: 424
Round: 425
Round: 426
Round: 427
Round: 428
Round: 429
Round: 430
Round: 431
Round: 432
Round: 433
Round: 434
Round: 435
Round: 436
Round: 437
Round: 438
Round: 439
Round: 440
Round: 441
Round: 442
Round: 443
Round: 444
Round: 445
Round: 446
Round: 447
Round: 448
Round: 449
Round: 450
Round: 451
Round: 452
Round: 453
Round: 454
Round: 455
Round: 456
Round: 457
Round: 458
Round: 459
Round: 460
Round: 461
Round: 462
Round: 463
Round: 464
Round: 465
Round: 466
Round: 467

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'log\\s382h1'

In [ ]:
X_df